In [1]:
import pandas as pd
import numpy as np
from dataAccessor import get_intraday, get_daily, get_singletime
# from config import av_api_key
from util import compare_df_cols, TIME_MAP, trading_datemap
from datetime import date
import pandas_market_calendars as mcal

In [8]:
class backTester():
    # order_book columns
    # order_id: id of trade, can have multiple arms via multiple instances of same id
    # ticker: ticker of the trade id's arm
    # order_type: buy or sell to open
    # order_value: the value of the trade at execute time
    # exec_date: date of exec
    # exec_time: time of exec, can be open/close or time in form hh:mm
    # exec_inftime: exec_time but inferred time for open/close mapped to hh:mm
    # exec_fulldate: combined exec_date and exec_inftime
    def __init__(self, order_book = None, stat_mode = None):
        
        self.orders = pd.DataFrame(columns=[
            'order_id',
            'ticker',
            'order_type',
            'order_value',
            'exec_date',
            'exec_time',
            'exec_inftime',
            'exec_fulldate'
        ])
        self.stat_mode = stat_mode

        self.MAX_TRADE_DAYS = 30
        
        if order_book: 
            self.add_orders(order_book)
            
    def add_orders(self,order_book):

        # generate exec_fulldate
        order_book['exec_inftime'] = order_book['exec_time'].map(TIME_MAP)
        # order_book['exec_fulldate'] = order_book['exec_date'] + order_book['exec_time'].map(TIME_MAP)
        order_book['exec_fulldate'] = order_book['exec_date'] + order_book['exec_inftime'].combine_first(order_book['exec_time'])
        
        # adds orders to order book
        a_not_b = compare_df_cols(self.orders, order_book)
        assert a_not_b.empty
        
        to_add = order_book[['order_id','order_type','order_value','exec_time','exec_date','ticker','exec_inftime','exec_fulldate']]
        self.orders = pd.concat([self.orders, order_book])

    def get_pfolio_exec(self, stat_mode):
        
        self.orders['exec_price']=self.orders.apply(
            lambda row: get_singletime(
                ticker=row['ticker'],date=row['exec_date'],dot=row['exec_time'],price_freq=stat_mode
            )['price'].values[0], 
            axis=1
        )

    def get_pfolio_fwd(self, stat_mode):
        if stat_mode == 'daily':

            # get forward prices of each order's ticker starting at exec date
            self.pfolio_prices = self.orders.copy()

            self.pfolio_prices['fwd'] = self.pfolio_prices.apply(
                lambda row: get_daily(
                    row['ticker'],
                    start_date=row['exec_date'],
                    end_date=row['exec_date']+pd.Timedelta(days=self.MAX_TRADE_DAYS)
                ), 
                axis = 1
            )

            # unnest, creates df of prices with associated order_id on each row
            fwd_prices = (
                pd.concat(self.pfolio_prices.set_index(self.orders.columns.tolist()).pop('fwd').to_dict())
                .rename_axis(tuple(self.orders.rename(columns={'ticker':'exec_ticker'}).columns.tolist() + ['new']))
                # # .rename_axis(('order_id','new'))
                .reset_index(level=[i for i in range(len(self.orders.columns.tolist()))])
                .reset_index(drop=True)
            )
            assert fwd_prices['exec_ticker'].equals(fwd_prices['ticker'])
            
            fwd_prices.drop(columns=['exec_ticker'],inplace=True)
            
            # changes open/close columns into one column called trade_price
            fwd_prices_open = fwd_prices[
                ['order_id','ticker','order_type','order_value','exec_date','exec_time','exec_inftime','exec_fulldate','exec_price','open','date']
            ]
            fwd_prices_open['fwd_inftime'] = pd.Timedelta(hours=9,minutes=30)
            fwd_prices_open['fwd_fulldate'] = fwd_prices_open['date'] + fwd_prices_open['fwd_inftime'] 
            fwd_prices_open.rename(columns={'date':'fwd_date','open':'fwd_price'}, inplace=True)

            fwd_prices_close = fwd_prices[
                ['order_id','ticker','order_type','order_value','exec_date','exec_time','exec_inftime','exec_fulldate','exec_price','close','date']
            ]
            fwd_prices_close['fwd_inftime'] = pd.Timedelta(hours=16)
            fwd_prices_close['fwd_fulldate'] = fwd_prices_close['date'] + fwd_prices_close['fwd_inftime']
            fwd_prices_close.rename(columns={'date':'fwd_date','close':'fwd_price'}, inplace=True)
            
            self.pfolio_prices = pd.concat([fwd_prices_open, fwd_prices_close])
            print(self.pfolio_prices.shape)
            self.pfolio_prices = self.pfolio_prices.loc[self.pfolio_prices.fwd_fulldate >= self.pfolio_prices.exec_fulldate]
            print(self.pfolio_prices.shape)
            
        elif stat_mode == 'intraday':
            raise NotImplementedError('TODO')

    def calc_pfolio_stats(self):
        # avg return over time
        # pct positive at time points
        # sharpe ratio of trades
        raise NotImplementedError('TODO')
        
    def trade(self, stat_mode = None):
        if stat_mode:
            self.stat_mode = stat_mode
        # get data
        if stat_mode == 'daily':
            # get exec prices
            self.get_pfolio_exec(self.stat_mode)
            # get forward prices
            self.get_pfolio_fwd(self.stat_mode)
        
        elif stat_mode == 'intraday':
            raise NotImplementedError('TODO')

        # calculate positions
        # self.pfolio_prices['fwd_cumu_unitval'] = self.pfolio_prices['fwd_price'] / self.pfolio_prices['exec_price']
        self.pfolio_prices['fwd_cumu_unitval'] = np.where(b.pfolio_prices['order_type'] == 'buy', b.pfolio_prices['fwd_price'] / b.pfolio_prices['exec_price'],
               np.where(b.pfolio_prices['order_type'] == 'sell', 2 - b.pfolio_prices['fwd_price']/b.pfolio_prices['exec_price'],np.nan))
        self.pfolio_prices['fwd_cumu_unitret'] = self.pfolio_prices['fwd_cumu_unitval'] - 1
        
        self.pfolio_prices['fwd_cumu_val'] = self.pfolio_prices['fwd_cumu_unitval'] * self.pfolio_prices['order_value']
        self.pfolio_prices['fwd_cumu_ret'] = self.pfolio_prices['fwd_cumu_unitret'] * self.pfolio_prices['order_value']
        
        t_datemap = trading_datemap(
            start_date = self.pfolio_prices.fwd_date.min(),
            end_date = self.pfolio_prices.fwd_date.max()
        )

        self.pfolio_prices['date_delta'] = ( 
            self.pfolio_prices['fwd_date'].dt.date.map(t_datemap) - self.pfolio_prices['exec_fulldate'].dt.date.map(t_datemap) 
        ) + (
            (self.pfolio_prices['fwd_inftime'] - self.pfolio_prices['exec_inftime']) / np.timedelta64(1, 'D')
        )

        # generate pfolio stats
    

    def results(self):
        raise NotImplementedError('TODO')


In [9]:
b = backTester()

In [10]:
test_orders = pd.DataFrame(
    [
        [1,'buy',1,'open',pd.to_datetime('2023-08-04'),'IBM'],
        [1,'buy',1,'close',pd.to_datetime('2022-03-08'),'GE'],
        [2,'sell',1,'open',pd.to_datetime('2023-08-04'),'^GSPC']
    ], 
    # index = [1, 2],
    columns=['order_id','order_type','order_value','exec_time','exec_date','ticker']
)
test_orders

,order_id,order_type,order_value,exec_time,exec_date,ticker
0,1,buy,1,open,2023-08-04,IBM
1,1,buy,1,close,2022-03-08,GE
2,2,sell,1,open,2023-08-04,^GSPC


In [11]:
b.add_orders(test_orders)

In [12]:
b.orders

,order_id,ticker,order_type,order_value,exec_date,exec_time,exec_inftime,exec_fulldate
0,1,IBM,buy,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00
1,1,GE,buy,1,2022-03-08,close,0 days 16:00:00,2022-03-08 16:00:00
2,2,^GSPC,sell,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00


In [13]:
b.trade(stat_mode = 'daily')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
(110, 13)
(109, 13)


C:\Users\danie\AppData\Local\Temp\ipykernel_31100\3139165496.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fwd_prices_open['fwd_inftime'] = pd.Timedelta(hours=9,minutes=30)
C:\Users\danie\AppData\Local\Temp\ipykernel_31100\3139165496.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fwd_prices_open['fwd_fulldate'] = fwd_prices_open['date'] + fwd_prices_open['fwd_inftime']
C:\Users\danie\AppData\Local\Temp\ipykernel_31100\3139165496.py:86: SettingWithCopyWarning: 
A value is trying to be set on a c

In [14]:
b.orders

,order_id,ticker,order_type,order_value,exec_date,exec_time,exec_inftime,exec_fulldate,exec_price
0,1,IBM,buy,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,145.089996
1,1,GE,buy,1,2022-03-08,close,0 days 16:00:00,2022-03-08 16:00:00,68.813431
2,2,^GSPC,sell,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,4513.959961


In [15]:
b.pfolio_prices

,order_id,ticker,order_type,order_value,exec_date,exec_time,exec_inftime,exec_fulldate,exec_price,fwd_price,fwd_date,fwd_inftime,fwd_fulldate,fwd_cumu_unitval,fwd_cumu_unitret,fwd_cumu_val,fwd_cumu_ret,date_delta
0,1,IBM,buy,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,145.089996,145.089996,2023-08-04,0 days 09:30:00,2023-08-04 09:30:00,1.000000,0.000000,1.000000,0.000000,0.000000
1,1,IBM,buy,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,145.089996,145.000000,2023-08-07,0 days 09:30:00,2023-08-07 09:30:00,0.999380,-0.000620,0.999380,-0.000620,1.000000
2,1,IBM,buy,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,145.089996,145.699997,2023-08-08,0 days 09:30:00,2023-08-08 09:30:00,1.004204,0.004204,1.004204,0.004204,2.000000
3,1,IBM,buy,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,145.089996,144.940002,2023-08-09,0 days 09:30:00,2023-08-09 09:30:00,0.998966,-0.001034,0.998966,-0.001034,3.000000
4,1,IBM,buy,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,145.089996,143.039993,2023-08-10,0 days 09:30:00,2023-08-10 09:30:00,0.985871,-0.014129,0.985871,-0.014129,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,2,^GSPC,sell,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,4513.959961,4399.770020,2023-08-21,0 days 16:00:00,2023-08-21 16:00:00,1.025297,0.025297,1.025297,0.025297,11.270833
51,2,^GSPC,sell,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,4513.959961,4387.549805,2023-08-22,0 days 16:00:00,2023-08-22 16:00:00,1.028004,0.028004,1.028004,0.028004,12.270833
52,2,^GSPC,sell,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,4513.959961,4436.009766,2023-08-23,0 days 16:00:00,2023-08-23 16:00:00,1.017269,0.017269,1.017269,0.017269,13.270833
53,2,^GSPC,sell,1,2023-08-04,open,0 days 09:30:00,2023-08-04 09:30:00,4513.959961,4376.310059,2023-08-24,0 days 16:00:00,2023-08-24 16:00:00,1.030494,0.030494,1.030494,0.030494,14.270833


In [22]:
b.pfolio_prices.loc[np.isclose(b.pfolio_prices['date_delta'], b.pfolio_prices['date_delta'].round(), atol=0.0001) ,['order_id','date_delta','fwd_cumu_ret']].groupby(['order_id','date_delta']).sum()['fwd_cumu_ret']

order_id  date_delta
1         0.0           0.000000
          1.0           0.034547
          2.0           0.040279
          3.0           0.045818
          4.0           0.034651
          5.0           0.034068
          6.0           0.060245
          7.0           0.058978
          8.0           0.061037
          9.0           0.047546
          10.0          0.041492
          11.0          0.037666
          12.0          0.050977
          13.0          0.043364
          14.0          0.032786
          15.0          0.067466
          16.0          0.072944
          17.0          0.038003
          18.0          0.049234
          19.0          0.043902
          20.0          0.022121
          21.0          0.019966
          22.0          0.014861
2         0.0           0.000000
          1.0           0.004958
          2.0           0.003529
          3.0           0.002745
          4.0           0.005937
          5.0           0.014017
          6.0         

In [23]:
b.pfolio_prices.to_csv('test.csv')